## Julia GASS

Testing the GASS with Julia

In [8]:
using PyCall
using Distributions
using Statistics

using Distances , Random 
using DataFrames

import CSV

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"

#push!(LOAD_PATH,"$rootdir/master/src")
#using GASS

import PyPlot

@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord

## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [9]:
## Types ##

struct _inputCfg
    file_parameters::String
    folder_results::String
    verbose::Bool
end

struct _observation
    Array_Configuration_File::String
    Observatory_Latitude::Float64
    Source_Declination::Float64
    Source_Hour_Angle::Float64
    Subarray_Number::Int
end

struct _subarrayParameters
        Pads_Per_Subarray::Vector{Int}
        Subarray_Name::Vector{String}
        Subrange::Array{UnitRange{Int64},1}
        Spatial_Resolution::Vector{Float64}
        Maximum_Recoverable_Scale::Vector{Float64}
        Elongation::Vector{Float64}
        Sidelobe_Level::Vector{Float64}
end

struct _weight
    Weight_Subarray::Vector{Float64}
    Weight_Spatial_Resolution::Vector{Float64}
    Weight_Maximum_Recoverable_Scale::Vector{Float64}
    Weight_Elongation::Vector{Float64}                  
    Weight_Sidelobe_Levels::Vector{Float64}
end

struct _GA
    Number_Iterations::Int
    Population_Size::Int
    Termination_Condition::Bool
    Threshold::Float64
    Mutation_Rate::Float64
    Tournament_Size::Int
    Number_Elitism::Int
end

struct _cfg
    arr::AbstractDataFrame 
    obs::_observation
    sub::_subarrayParameters
    wei::_weight
    ga::_GA
    inp::_inputCfg
end

In [10]:
function _input_parameters(file)
    let
        param = []
        open(file,"r") do f
            for line in eachline(f)
                l = lstrip(line)
                if length(l) > 0 && l[1] != '#'
                    res = strip.(split(l,":"))
                    push!(param,res)
                end
            end
        end
        return(param)
    end
end

_input_parameters (generic function with 1 method)

In [11]:
## parse input file

function _parse_input(input_param)
    let
        fileParam= "default.cfg" 
        saveFold=  "Results/"
        verbose= true
        
        for pair in input_param
            if pair[1] == "File_Parameters"
                fileParam = pair[2]
            elseif pair[1] == "Folder_Results"
                saveFold= pair[2]
            elseif pair[1] == "Display_Screen_Results" && pair[2] == "false"
                verbose= false
            end
        end
        res= _inputCfg(fileParam , saveFold , verbose)
        return(res)    
    end
end

_parse_input (generic function with 1 method)

In [12]:
## parse parameter file for the subarray constraints

function _parse_parameters(input_param , inpcfg)
    let
        arraycfg= 0 
        obs= 0
        sub= 0
        wei= 0
        ga= 0
        
        Array_Configuration_File= "alma.cfg"
        Observatory_Latitude= -23.0262015
        Source_Declination= -30
        Source_Hour_Angle = -1 
        Subarray_Number= 4
        
        ## array of nsubarray elements
        Pads_Per_Subarray= []
        Weight_Subarray= []
        Subarray_Name= [] 
        Spatial_Resolution= []
        Maximum_Recoverable_Scale=
        Elongation= []
        Sidelobe_Level= []
        Weight_Spatial_Resolution= []
        Weight_Maximum_Recoverable_Scale= [] 
        Weight_Elongation= []                    
        Weight_Sidelobe_Levels= []
        
        ## GA parameters
        Number_Iterations = 100
        Population_Size= 100
        Termination_Condition= false
        Threshold= -0.05
        Mutation_Rate= 0.05
        Tournament_Size= 5
        Number_Elitism= 5
        
        for pair in input_param
            if pair[1] == "Array_Configuration_File"
                Array_Configuration_File= pair[2]
            elseif pair[1] == "Observatory_Latitude"
                Observatory_Latitude= parse(Float64, pair[2])    
            elseif pair[1] == "Source_Declination"
                Source_Declination= parse(Float64, pair[2])
            elseif pair[1] == "Source_Hour_Angle"
                Source_Hour_Angle= parse(Float64, pair[2])
            elseif pair[1] == Subarray_Number
                Subarray_Number= parse(Int, pair[2])
            elseif pair[1] == "Number_Iterations"
                Number_Iterations= parse(Int, pair[2])
            elseif pair[1] == "Population_Size"
                Population_Size= parse(Int, pair[2])
            elseif pair[1] == "Termination_Condition"
                Termination_Condition= pair[2]=="true" ? true : false
            elseif pair[1] == "Threshold"
                Threshold=parse(Float64, pair[2]) 
            elseif pair[1] == "Mutation_Rate"
                Mutation_Rate=parse(Float64, pair[2])
            elseif pair[1] == "Tournament_Size"
                Tournament_Size= parse(Int, pair[2])
            elseif pair[1] == "Number_Elitism"
                Number_Elitism= parse(Int, pair[2])  
            elseif pair[1] == "Pads_Per_Subarray"
                Pads_Per_Subarray= map(x->(v = tryparse(Int,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weights_Subarray"
                  Weight_Subarray= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))  
            elseif pair[1] == "Subarray_Name" 
                Subarray_Name= split(pair[2],",")
            elseif pair[1] == "Spatial_Resolution"
                Spatial_Resolution= map(x->(v = tryparse(Float64,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Maximum_Recoverable_Scale"
                Maximum_Recoverable_Scale= map(x->(v = tryparse(Float64,x); 
                        v==nothing ? 0.0 : v),split(pair[2],","))       
            elseif pair[1] == "Elongation"
                Elongation= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Sidelobe_Level"
               Sidelobe_Level= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],",")) 
            elseif pair[1] == "Weight_Spatial_Resolution"
                Weight_Spatial_Resolution= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Maximum_Recoverable_Scale" 
                Weight_Maximum_Recoverable_Scale= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Elongation" 
                Weight_Elongation= map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            elseif pair[1] == "Weight_Sidelobe_Levels"
                Weight_Sidelobe_Levels=map(x->(v = tryparse(Float64,x);
                        v==nothing ? 0.0 : v),split(pair[2],","))
            end
        end
        
        ###
        println(typeof(Array_Configuration_File))
        
        arrcfg = CSV.read(convert(String,Array_Configuration_File), datarow=4 , header=["X" , "Y", "Z" , "diam" , "name"] , delim= " ")
           
        subrange = []
        start= 1
            for npad in Pads_Per_Subarray
                push!(subrange,start:start+npad-1)
                start= start+npad
            end
        
        ### setting the struct.
        obs= _observation(Array_Configuration_File, Observatory_Latitude , Source_Declination,Source_Hour_Angle,
            Subarray_Number)
        sub= _subarrayParameters(Pads_Per_Subarray, Subarray_Name , subrange, Spatial_Resolution,
            Maximum_Recoverable_Scale , Elongation, Sidelobe_Level)
        wei= _weight(Weight_Subarray,Weight_Spatial_Resolution,Weight_Maximum_Recoverable_Scale, 
            Weight_Elongation,Weight_Sidelobe_Levels)
        ga= _GA(Number_Iterations , Population_Size,Termination_Condition ,Threshold ,
            Mutation_Rate , Tournament_Size ,Number_Elitism)
        
        
        ### population setting
        pop =_cfg(arrcfg , obs , sub , wei , ga , inpcfg)
        
   return(pop)     
end
end



_parse_parameters (generic function with 1 method)

In [13]:
function _check_consistency(pop::_cfg)
    nsubpads= sum(pop.sub.Pads_Per_Subarray)
    npads= nrow(pop.arr)
    if nsubpads != npads
        error("##Error: Number of pads per subarray is not equal to the total of pads.")
    end
    return(true)
end

## main function to read the cfg and check it.
function _read_cfg(inpfile)
    res= _input_parameters(inpfile)
    inpcfg= _parse_input(res)
    ## parameters inputs
    res= _input_parameters(inpcfg.file_parameters)
    cfg= _parse_parameters(res, inpcfg)
    _check_consistency(cfg)
    return(cfg)
end


_read_cfg (generic function with 1 method)

In [14]:
macro main(inpfile)
    cfg= _read_cfg(inpfile)
end

@main("../master/data/GA_Inputs_O-10.txt.julia")
# @main("../master/data/GA_Parameters_O-10.txt.julia")

SubString{String}


_cfg(50×5 DataFrame
│ Row │ X        │ Y        │ Z         │ diam     │ name    │
│     │ Float64⍰ │ Float64⍰ │ Float64⍰  │ Float64⍰ │ String⍰ │
├─────┼──────────┼──────────┼───────────┼──────────┼─────────┤
│ 1   │ -33.8941 │ -712.752 │ -2.33009  │ 12.0     │ A001    │
│ 2   │ -22.5589 │ -691.984 │ -2.32374  │ 12.0     │ A003    │
│ 3   │ -9.8351  │ -663.827 │ -2.7227   │ 12.0     │ A009    │
│ 4   │ 10.7399  │ -659.567 │ -2.72626  │ 12.0     │ A015    │
│ 5   │ -20.6204 │ -633.907 │ -2.52767  │ 12.0     │ A016    │
│ 6   │ -41.2393 │ -725.978 │ -2.33663  │ 12.0     │ A018    │
│ 7   │ -61.0568 │ -722.793 │ -1.93443  │ 12.0     │ A020    │
│ 8   │ -86.8729 │ -732.262 │ -1.92881  │ 12.0     │ A026    │
│ 9   │ -103.296 │ -719.779 │ -1.93063  │ 12.0     │ A032    │
│ 10  │ -105.439 │ -755.003 │ -1.64075  │ 12.0     │ A033    │
⋮
│ 40  │ -320.565 │ -763.861 │ 2.92101   │ 12.0     │ A086    │
│ 41  │ 210.998  │ -883.466 │ -6.96694  │ 12.0     │ A087    │
│ 42  │ -71.1672 │ -427.394 │ -0.